In [2]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

In [ ]:
TJData.csv['processo_unico']

In [ ]:
len(processos)

In [3]:
num_lines = sum(1 for line in open(os.path.join('/tj_files',TJData.csv['processo_unico']),encoding='latin1'))

In [ ]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias = list(map(lambda x : x,serventias_jec.cod_serv))
serventias.append(500001)

In [ ]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec = list(map(lambda x : x,serventias_jec.cod_serv))
# colunas que serão utilizadas no processo
usecols=['COD_PROC','COD_COMP','COD_SERV','ID_PROC','COD_ASSUNTO','DATA_CAD']

# obtem uma lista com tuplas combinando as serventias e o intervalo de ano desejado
servs_anos = TJData.build_serventias_anos_list(list(serventias_jec.cod_serv),range(2013,2017))

# obtem a lista de arquivos disponíveis para processar
files_processo = TJData.list_files_in_serventias_anos(TJData.processo,servs_anos)
# le em paralelo os arquivos referentes aos processos
processo_dfs = TJData.concurrent_read(TJData.clean_processo, files_processo, kwargs = {
    'usecols':usecols
})
processo_dfs = pd.concat(processo_dfs)

In [6]:
df = TJData.read_csv(TJData.csv['processo_unico'], nrows=3000000)

In [ ]:
df = df.rename(columns=lambda x : x.lower())
df.id_proc = df.id_proc.apply(lambda x : str(x))
processo_dfs = processo_dfs.merge(cnk1, on=['cod_proc','id_proc'], how='left')

In [ ]:
df = df[df.serventia.isin(cod_procs)]

In [ ]:
serventias = list(map(lambda x : x,serventias_jec.cod_serv))

In [ ]:
serventias = r''.join(str(serv)+'|' for serv in serventias)[:-1]

In [ ]:
df[df.COD_PROC.str.extract(fr'\d{4}.\d{3}.(\d{6})-\d\w?').]

In [ ]:
fr'\d{4}.\d{3}.({serventias})-\d\w?'

In [ ]:
df[df.COD_PROC.str.extract(r'\d{4}.\d{3}.(\d{6})-\d\w?')..str.match(serventias)]

In [ ]:
df.cod_proc_serv = df.COD_PROC.str.extract(r'\d{4}.\d{3}.(\d{6})-\d\w?',expand=True)

In [ ]:
df = df.assign(serventia = df.COD_PROC.str.extract(r'\d{4}.\d{3}.(\d{6})-\d\w?'))
df.serventia = df.cod_proc_serv.astype(int)
df = df[df.serventia.isin(serventias)]
df

In [7]:
df

,ID_PROC,COD_CNJ,COD_PROC,COD_TIP_PROC,IND_EXIB_COD_PROC
0,511,0000019-45.1901.8.19.0204,1901.204.000019-9,1,S
1,512,0000001-17.1902.8.19.0001,1902.001.500001-5,1,S
2,513,0000002-02.1902.8.19.0001,1902.001.500002-7,1,S
3,514,0000001-66.1902.8.19.0017,1902.017.000001-5,1,S
4,515,0000002-51.1902.8.19.0017,1902.017.000002-7,1,S
5,516,0000003-36.1902.8.19.0017,1902.017.000003-9,1,S
6,517,0000004-21.1902.8.19.0017,1902.017.000004-0,1,S
7,518,0000005-06.1902.8.19.0017,1902.017.000005-2,1,S
8,519,0000006-88.1902.8.19.0017,1902.017.000006-4,1,S
9,520,0000007-73.1902.8.19.0017,1902.017.000007-6,1,S


In [5]:
num_lines/1500000

2.913408